In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import walk
import re

# Cars Data for Classification Problem

In [6]:
names_file = [line.strip() for line in open('./car.names', 'r').readlines()]

In [7]:
names_file

['1. Title: Car Evaluation Database',
 '',
 '2. Sources:',
 '(a) Creator: Marko Bohanec',
 '(b) Donors: Marko Bohanec   (marko.bohanec@ijs.si)',
 'Blaz Zupan      (blaz.zupan@ijs.si)',
 '(c) Date: June, 1997',
 '',
 '3. Past Usage:',
 '',
 'The hierarchical decision model, from which this dataset is',
 'derived, was first presented in',
 '',
 'M. Bohanec and V. Rajkovic: Knowledge acquisition and explanation for',
 'multi-attribute decision making. In 8th Intl Workshop on Expert',
 'Systems and their Applications, Avignon, France. pages 59-78, 1988.',
 '',
 'Within machine-learning, this dataset was used for the evaluation',
 'of HINT (Hierarchy INduction Tool), which was proved to be able to',
 'completely reconstruct the original hierarchical model. This,',
 'together with a comparison with C4.5, is presented in',
 '',
 'B. Zupan, M. Bohanec, I. Bratko, J. Demsar: Machine learning by',
 'function decomposition. ICML-97, Nashville, TN. 1997 (to appear)',
 '',
 '4. Relevant Information

In [11]:
df = pd.read_csv('car.data')

In [12]:
df.columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc
...,...,...,...,...,...,...,...
1722,low,low,5more,more,med,med,good
1723,low,low,5more,more,med,high,vgood
1724,low,low,5more,more,big,low,unacc
1725,low,low,5more,more,big,med,good


# Test and Train Split with Scikit-learn

In [15]:
from sklearn.model_selection import train_test_split
y = pd.DataFrame(df['class'])
X = df.drop(['class'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [17]:
#check the object types
X_train.dtypes

buying      object
maint       object
doors       object
persons     object
lug_boot    object
safety      object
dtype: object

In [18]:
# use scikit-learn build in function for label encoding this object type values

In [20]:
from sklearn.preprocessing import LabelEncoder


label_X_train = X_train.copy()
label_X_test = X_test.copy()
label_y_train = y_train.copy()
label_y_test = y_test.copy()

label_encoder = LabelEncoder()
for col in X_train.columns:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_test[col] = label_encoder.transform(X_test[col])

X_train = label_X_train
X_test = label_X_test    

for col in y_train.columns:
    label_y_train[col] = label_encoder.fit_transform(y_train[col])
    label_y_test[col] = label_encoder.transform(y_test[col])
    
y_train = label_y_train
y_test = label_y_test

# Decision Tree

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [23]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [26]:
pred = clf.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96       118
           1       0.78      0.82      0.80        17
           2       0.99      1.00      1.00       361
           3       0.88      0.91      0.89        23

    accuracy                           0.98       519
   macro avg       0.91      0.92      0.91       519
weighted avg       0.98      0.98      0.98       519



# Naive Bayes 

In [27]:
from sklearn.naive_bayes import BaseNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import CategoricalNB

In [29]:
clf2 = CategoricalNB()
clf2.fit(X_train, y_train)

/home/yazdan/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


CategoricalNB()

In [30]:
pred = clf2.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.65      0.61      0.63       118
           1       0.45      0.29      0.36        17
           2       0.90      0.97      0.93       361
           3       0.86      0.26      0.40        23

    accuracy                           0.83       519
   macro avg       0.72      0.53      0.58       519
weighted avg       0.82      0.83      0.82       519



In [31]:
clf3 = MultinomialNB()
clf3.fit(X_train, y_train)

/home/yazdan/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


MultinomialNB()

In [32]:
pred = clf3.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      0.02      0.03       118
           1       0.00      0.00      0.00        17
           2       0.70      1.00      0.82       361
           3       0.00      0.00      0.00        23

    accuracy                           0.70       519
   macro avg       0.42      0.25      0.21       519
weighted avg       0.71      0.70      0.58       519



/home/yazdan/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Neural Network

In [33]:
from sklearn.neural_network import MLPClassifier

In [67]:
nn_clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
nn_clf.fit(X_train, y_train)

pred = nn_clf.predict(X_test)

/home/yazdan/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/yazdan/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [68]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.77      0.82      0.80       118
           1       0.09      0.06      0.07        17
           2       0.96      0.96      0.96       361
           3       0.77      0.74      0.76        23

    accuracy                           0.89       519
   macro avg       0.65      0.65      0.65       519
weighted avg       0.88      0.89      0.89       519



In [69]:
nn_clf2 = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=42)
nn_clf2.fit(X_train, y_train)

pred2 = nn_clf.predict(X_test)

/home/yazdan/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/yazdan/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [70]:
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

           0       0.77      0.82      0.80       118
           1       0.09      0.06      0.07        17
           2       0.96      0.96      0.96       361
           3       0.77      0.74      0.76        23

    accuracy                           0.89       519
   macro avg       0.65      0.65      0.65       519
weighted avg       0.88      0.89      0.89       519



In [71]:
nn_clf3 = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
nn_clf3.fit(X_train, y_train)

pred3 = nn_clf.predict(X_test)

/home/yazdan/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/yazdan/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [72]:
print(classification_report(y_test, pred3))

              precision    recall  f1-score   support

           0       0.77      0.82      0.80       118
           1       0.09      0.06      0.07        17
           2       0.96      0.96      0.96       361
           3       0.77      0.74      0.76        23

    accuracy                           0.89       519
   macro avg       0.65      0.65      0.65       519
weighted avg       0.88      0.89      0.89       519

